# Arizona Scraping 

In [21]:
# set up 
import pandas as pd 
import requests 
import bs4
import numpy as np
import time 
import re
from tika import parser
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from io import BytesIO
from urllib.request import urlopen

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

%xmode Minimal

Exception reporting mode: Minimal


In [2]:
# # set headers
# header_list = {'User-Agent': REDACTED,
#               'Accept-Language': 'en-US,en;q=0.9'}

In [3]:
# review of robots.txt - 10 second crawl delay

## Scrape Links
`Selenium`

### Test on one link

In [17]:
# access webpage
link = 'https://azgovernor.gov/news-releases'
server_response = requests.get(link, header_list)
server_response

<Response [200]>

In [20]:
# parse and isolate content
soup = bs4.BeautifulSoup(server_response.content, features="html.parser")
#soup
results = soup.find_all('div', class_ = 'item-box-desc') 
# results
title = [i.find('a')['title'] for i in results]
# title
# title = [i.replace(u'\ufeff', u'') for i in title]
# title 
date = [i.find('div', class_ = 'date col-xs-12 padding6').text for i in results]
# date
link = [i.find('a')['href'] for i in results]
# link

['Governor Katie Hobbs and Secretary of State Adrian Fontes Call for Clean Fix to Election Timeline',
 'Governor Hobbs Announces Appointment of Joshua Steinlage to Coconino County Superior Court',
 'ICYMI: Governor Katie Hobbs Unveils Plan That Will “Help Everyday Families and Arizonans Save on Prescription Drugs”',
 'Governor Katie Hobbs Joins Reproductive Freedom Alliance in Amicus Brief to Defend Arizonans’ Access to Safe Reproductive Healthcare',
 'Governor Hobbs Appoints Members to the Commission of African American Affairs',
 'Governor Katie Hobbs Joins Senate Democratic Leader Epstein, House Democratic Leader Contreras in Introducing ESA Accountability Legislation',
 'Governor Katie Hobbs Joins Rep. Hamilton, Sen. Sundareshan Introducing Legislation to Protect Reproductive Healthcare']

['February 5, 2024',
 'February 1, 2024',
 'February 1, 2024',
 'January 30, 2024',
 'January 30, 2024',
 'January 29, 2024',
 'January 24, 2024']

['https://azgovernor.gov/office-arizona-governor/news/2024/02/governor-katie-hobbs-and-secretary-state-adrian-fontes-call',
 'https://azgovernor.gov/office-arizona-governor/news/2024/02/governor-hobbs-announces-appointment-joshua-steinlage-coconino',
 'https://azgovernor.gov/office-arizona-governor/news/2024/02/icymi-governor-katie-hobbs-unveils-plan-will-help-everyday',
 'https://azgovernor.gov/office-arizona-governor/news/2024/01/governor-katie-hobbs-joins-reproductive-freedom-alliance-amicus',
 'https://azgovernor.gov/office-arizona-governor/news/2024/01/governor-hobbs-appoints-members-commission-african-american',
 'https://azgovernor.gov/office-arizona-governor/news/2024/01/governor-katie-hobbs-joins-senate-democratic-leader-epstein',
 'https://azgovernor.gov/office-arizona-governor/news/2024/01/governor-katie-hobbs-joins-rep-hamilton-sen-sundareshan']

### Scrape all link

In [59]:
# try selenium for scraping since it has a continuous scroll
driver = webdriver.Chrome()
driver.get("https://azgovernor.gov/news-releases")

def scroll_down():
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait for to load more content (if any)
    driver.implicitly_wait(20)

# Scroll down multiple times to load more content
for _ in range(25):  
    scroll_down()
    
#     wait = WebDriverWait(driver, 10)
    results = driver.find_elements(By.CLASS_NAME,'item-box-desc')
    results_list = [{'title': i.find_element(By.TAG_NAME, 'a').get_attribute('title'),
                 'date': i.find_element(By.TAG_NAME, 'div').text, 
                'link': i.find_element(By.TAG_NAME, 'a').get_attribute('href')} for i in results]
    df = pd.DataFrame.from_dict(results_list)
    
driver.close()

In [60]:
# add blank content column for later
df['content'] = np.nan
df.head()

,title,date,link,content
0,Governor Katie Hobbs and Secretary of State Ad...,"February 5, 2024",https://azgovernor.gov/office-arizona-governor...,NaN
1,Governor Hobbs Announces Appointment of Joshua...,"February 1, 2024",https://azgovernor.gov/office-arizona-governor...,NaN
2,ICYMI: Governor Katie Hobbs Unveils Plan That ...,"February 1, 2024",https://azgovernor.gov/office-arizona-governor...,NaN
3,Governor Katie Hobbs Joins Reproductive Freedo...,"January 30, 2024",https://azgovernor.gov/office-arizona-governor...,NaN
4,Governor Hobbs Appoints Members to the Commiss...,"January 30, 2024",https://azgovernor.gov/office-arizona-governor...,NaN
...,...,...,...,...
163,Katie Hobbs Sworn In As Arizona’s 24th Governor,"January 2, 2023",https://azgovernor.gov/office-arizona-governor...,NaN
164,Governor-Elect Katie Hobbs Announces Public Sa...,"December 29, 2022",https://azgovernor.gov/office-arizona-governor...,NaN
165,Governor-Elect Katie Hobbs Announces Environme...,"December 28, 2022",https://azgovernor.gov/office-arizona-governor...,NaN
166,Governor-Elect Katie Hobbs Announces Health an...,"December 27, 2022",https://azgovernor.gov/office-arizona-governor...,NaN


In [61]:
df.to_csv('arizona_links.csv', index = False)

## Scrape content from links
`Beautiful Soup`

In [62]:
# define link scraping 
def get_arizona_content(link, df, header_list):
    try: 
        # check if content has already been pulled
        result = df.loc[df.link == link, 'content'].notnull().all()
        if result:
            return f'content already existing for {link}' 
        else: 
            try: 
                df['content']
                server_response = requests.get(link, headers = header_list)
                soup_link = bs4.BeautifulSoup(server_response.content, features="html.parser")
                result = soup_link.find('div', class_ = 'margin-top20 clearfix field-name-body margin-bottom20').find_all('p')
                string_result = [str(i.text) for i in result]
                string_result_clean = [i.replace(u'\xa0', u' ') for i in string_result]
                result_merged = " ".join(string_result_clean)
                df.loc[df.link == link, 'content'] = result_merged
                return f"success: content added from {link}"
            except: 
                pass
    except Exception as e: 
        return f"unable to gather content from {link} due to {e}"
    #return df_updated 

### Test for one link

In [48]:
# test for one link 
link = 'https://gov.alaska.gov/governor-dunleavy-creates-alaska-energy-security-task-force-to-create-comprehensive-statewide-energy-plan/'
server_response = requests.get(link, headers = header_list)
server_response

<Response [200]>

In [49]:
# Parse and isolate content
soup_link = bs4.BeautifulSoup(server_response.content, features="html.parser")
result = soup_link.find('div', class_ = 'et_pb_module et_pb_post_content et_pb_post_content_0_tb_body').find_all('p')
string_result = [str(i.text) for i in result]
string_result_clean = [i.replace(u'\xa0', u' ') for i in string_result]
result_merged = " ".join(string_result_clean)
result_merged

'Today Governor Mike Dunleavy issued Administrative Order 344, establishing the Alaska Energy Security Task Force. The purpose of the task force is to develop a comprehensive statewide energy plan that will evaluate the energy generation, distribution, and transmission for the State of Alaska and its communities. The development of the plan will include collaborating with both the public and private stakeholders. The statewide energy plan, including proposed timelines and milestones, will be presented to the Governor upon completion. “Despite Alaska’s position as a leading producer of energy, the cost of energy in Alaska, especially in our rural communities, is extremely high,” said Governor Mike Dunleavy. “As everyone has been reminded by the war in Ukraine, access to and cost of energy are influenced by global events. I’m establishing this task force to create a plan that will reduce Alaska’s vulnerability to fluctuating energy markets by securing dependable and affordable energy for

## Scrape content for all links

In [63]:
# test on 5 to see if it works
for i in df.link[0:6]:
     get_arizona_content(i, df, header_list)
        
df.head()

'success: content added from https://azgovernor.gov/office-arizona-governor/news/2024/02/governor-katie-hobbs-and-secretary-state-adrian-fontes-call'

'success: content added from https://azgovernor.gov/office-arizona-governor/news/2024/02/governor-hobbs-announces-appointment-joshua-steinlage-coconino'

'success: content added from https://azgovernor.gov/office-arizona-governor/news/2024/02/icymi-governor-katie-hobbs-unveils-plan-will-help-everyday'

'success: content added from https://azgovernor.gov/office-arizona-governor/news/2024/01/governor-katie-hobbs-joins-reproductive-freedom-alliance-amicus'

'success: content added from https://azgovernor.gov/office-arizona-governor/news/2024/01/governor-hobbs-appoints-members-commission-african-american'

'success: content added from https://azgovernor.gov/office-arizona-governor/news/2024/01/governor-katie-hobbs-joins-senate-democratic-leader-epstein'

AttributeError: 'DataFrame' object has no attribute 'header'

In [65]:
# run for all
for i in df.link:
     get_arizona_content(i, df, header_list)

'content already existing for https://azgovernor.gov/office-arizona-governor/news/2024/02/governor-katie-hobbs-and-secretary-state-adrian-fontes-call'

'content already existing for https://azgovernor.gov/office-arizona-governor/news/2024/02/governor-hobbs-announces-appointment-joshua-steinlage-coconino'

'content already existing for https://azgovernor.gov/office-arizona-governor/news/2024/02/icymi-governor-katie-hobbs-unveils-plan-will-help-everyday'

'content already existing for https://azgovernor.gov/office-arizona-governor/news/2024/01/governor-katie-hobbs-joins-reproductive-freedom-alliance-amicus'

'content already existing for https://azgovernor.gov/office-arizona-governor/news/2024/01/governor-hobbs-appoints-members-commission-african-american'

...

'success: content added from https://azgovernor.gov/office-arizona-governor/news/2023/01/katie-hobbs-sworn-arizonas-24th-governor'

'success: content added from https://azgovernor.gov/office-arizona-governor/news/2022/12/governor-elect-katie-hobbs-announces-public-safety-cabinet'

'success: content added from https://azgovernor.gov/office-arizona-governor/news/2022/12/governor-elect-katie-hobbs-announces-environmental-and-natural'

'success: content added from https://azgovernor.gov/office-arizona-governor/news/2022/12/governor-elect-katie-hobbs-announces-health-and-human-services'

'success: content added from https://azgovernor.gov/office-arizona-governor/news/2022/12/governor-elect-katie-hobbs-announces-additional-key-roles'

## Validate and Clean

In [67]:
#check that content was pulled accurately
missing = df[df.content.isna()]
missing 

,title,date,link,content


In [69]:
# clean data column
df['date_clean'] = pd.to_datetime(df['date'])
df_clean = df.drop(columns='date')

In [71]:
# subset to only 2023 dates
arizona_2023 = df_clean[(df_clean['date_clean'] >= '2023-01-01')
                     & (df_clean['date_clean'] <= '2023-12-31')].copy()

In [72]:
print(f"pulled {str(len(arizona_2023))} documents from arizona for 2023")

pulled 150 documents from arizona for 2023


## Export

In [73]:
# export
arizona_2023.to_csv('arizona_2023.csv', index = False)

In [75]:
arizona_2023.head()

,title,link,content,date_clean
14,Governor’s Office Demands ESA Transparency and...,https://azgovernor.gov/office-arizona-governor...,ESA program has surpassed expected budget esti...,2023-12-21
15,Governor Katie Hobbs Signs Executive Order Sen...,https://azgovernor.gov/office-arizona-governor...,Guard to Assist DPS and Local Law Enforcement ...,2023-12-15
16,Governor Katie Hobbs Announces Federal Grant t...,https://azgovernor.gov/office-arizona-governor...,"Phoenix, AZ – Today, Governor Katie Hobbs anno...",2023-12-08
17,Governor Katie Hobbs Launches Operation SECURE...,https://azgovernor.gov/office-arizona-governor...,State Charges Federal Government for Costs of ...,2023-12-08
18,"Sinema, Kelly, Hobbs Statement on Lukeville PO...",https://azgovernor.gov/office-arizona-governor...,ARIZONA – Arizona Senators Kyrsten Sinema and ...,2023-12-01
